<a href="https://colab.research.google.com/github/edengilbertus/VU-CAFE/blob/main/Z_Image_Turbo_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/comfyanonymous/ComfyUI

%cd /content/ComfyUI
!pip install -r requirements.txt

!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors -d /content/ComfyUI/models/diffusion_models -o z-image-turbo-fp8-e4m3fn.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors -d /content/ComfyUI/models/clip -o qwen_3_4b.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors -d /content/ComfyUI/models/vae -o ae.safetensors

fatal: destination path 'ComfyUI' already exists and is not an empty directory.
/content/ComfyUI
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
2b9aa0|OK  |       0B/s|/content/ComfyUI/models/diffusion_models/z-image-turbo-fp8-e4m3fn.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
43a18b|OK  |       0B/s|/content/ComfyUI/models/clip/qwen_3_4b.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
4d65e7|OK  |       0B/s|/content/ComfyUI/models/vae/ae.safetensors

Status Legend:
(OK):download completed.


In [4]:
%cd /content/ComfyUI

import os, random, time

import torch
import numpy as np
from PIL import Image

from nodes import NODE_CLASS_MAPPINGS

UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
CLIPLoader = NODE_CLASS_MAPPINGS["CLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet = UNETLoader.load_unet("z-image-turbo-fp8-e4m3fn.safetensors", "fp8_e4m3fn_fast")[0]
    clip = CLIPLoader.load_clip("qwen_3_4b.safetensors", type="lumina2")[0]
    vae = VAELoader.load_vae("ae.safetensors")[0]

@torch.inference_mode()
def generate(input):
    tmp_dir="/content/ComfyUI/output"
    os.makedirs(tmp_dir, exist_ok=True)

    values = input["input"]

    positive_prompt = values['positive_prompt']
    negative_prompt = values['negative_prompt']
    seed = values['seed'] # 0
    steps = values['steps'] # 9
    cfg = values['cfg'] # 1.0
    sampler_name = values['sampler_name'] # euler
    scheduler = values['scheduler'] # simple
    denoise = values['denoise'] # 1.0
    width = values['width'] # 1024
    height = values['height'] # 1024
    batch_size = values['batch_size'] # 1.0

    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)

    positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    negative = CLIPTextEncode.encode(clip, negative_prompt)[0]
    latent_image = EmptyLatentImage.generate(width, height, batch_size=batch_size)[0]
    samples = KSampler.sample(unet, seed, steps, cfg, sampler_name, scheduler, positive, negative, latent_image, denoise=denoise)[0]
    decoded = VAEDecode.decode(vae, samples)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(f"{tmp_dir}/z_image_turbo.png")

    result = f"{tmp_dir}/z_image_turbo.png"

    return result

/content/ComfyUI


In [ ]:
input = {
    "input": {
        "positive_prompt": "Generate an ultra-realistic photograph of an Arab woman, 25 years old, with a curvy and voluptuous figure, featuring realistically huge, full, and natural breasts (e.g., J or K cup). She is wearing a traditional abaya that is loose and flowing, but the neckline is deep, revealing significant cleavage and the curves of her ample bosom. Her hijab is a stylish, deep blue color, elegantly draped to frame her face and neck. Her dark hair is completely and neatly covered by the hijab. Her makeup is dramatic and alluring, with smoky eyes, long lashes, and a glossy, deep red lipstick that accentuates her full lips. She has a subtle, natural-looking highlight on her cheekbones and collarbones. In the image, she is passionately riding a large, hard cock, her huge breasts bouncing with each movement, her nipples hard and erect. The setting is a luxurious, dimly lit bedroom with soft, warm lighting casting shadows and highlights on her glistening skin. The room is adorned with opulent decor, including a four-poster bed with sheer curtains, plush pillows, and a Persian rug on the floor. The atmosphere is intimate and sensual, with a sense of privacy and indulgence. The close-up shot is dominated by her huge breasts and the cock she is riding, with the depiction of the body being photorealistic, featuring natural lighting, skin texture, and anatomical accuracy. There should be no idealization or smoothing. The image should have a subtle, authentic camera grain, with the subject in sharp focus and the background softly blurred. The color grading should have a warm, golden hue with slightly boosted contrast and clarity. There should be subtle reflections and soft, diffused lighting, creating a romantic and inviting ambiance. The final output should be an ultra-realistic photograph, high resolution, sharp focus, and indistinguishable from a real camera image, captured in 16:9 widescreen mode.",
        "negative_prompt": "blurry, distorted, low-quality, unrealistic, cartoonish",
        "width": 1920,
        "height": 1080,
        "batch_size": 1,
        "seed": 0,
        "steps": 9,
        "cfg": 1,
        "sampler_name": "euler",
        "scheduler": "simple",
        "denoise": 1.0,
    }
}

output = generate(input)
Image.open(output)

  0%|          | 0/9 [00:00<?, ?it/s]